In [14]:
import tensorflow as tf
import numpy as np
from src.ml.model import get_model

In [5]:
current_model = get_model()
current_model.load_weights('robot_reboot_model/model_0/')

In [23]:
trained_model = get_model()
trained_model.load_weights('robot_reboot_model/model_0/')

In [19]:
from src.alphazero.heuristic_function import heuristic_fn
from src.alphazero.montecarlo_tree_search import MonteCarloTreeSearch
from src.robot_reboot.factory import RobotRebootFactory
from src.robot_reboot.model import RobotRebootModel
from src.alphazero.game_player import GamePlayer


rows, cols, layers = 31, 31, 9

def build_mcts(nn, game, max_depth, playouts):
    model = RobotRebootModel(game, nn)
    game_player = GamePlayer(model, game)
    mcts = MonteCarloTreeSearch(heuristic_fn, max_depth, game_player, playouts=playouts)
    return mcts, game
    
def play(state, mcts, game, depth=0, max_depth=2):
    if depth > max_depth:
        return state
    p = mcts.search(state)
    i_best = np.argsort(p)[::-1][0]
    action = game.actions[i_best]
    next_state = game.apply(action, state)
    return play(next_state, mcts, game, depth=depth + 1, max_depth=max_depth)

def evaluate(nn, state, i, max_depth =20, playouts=50):
    mcts, game = build_mcts(nn, max_depth, playouts)
    final_state = play(state, mcts, game, max_depth=max_depth)
    v = game.get_value(final_state)
    score = game.get_score(final_state)
    
    sample = tf.train.Example(
        features=tf.train.Features(
            feature={
                'v': tf.train.Feature(float_list=tf.train.FloatList(value=[v])),
                'score': tf.train.Feature(float_list=tf.train.FloatList(value=[score])),
                's': tf.train.Feature(float_list=tf.train.FloatList(value=final_state.reshape(rows * cols * layers, )))
            }
        )
    )
    print(f'{i}: score {score} and v {v}')
    with tf.io.TFRecordWriter(f'results/{i}_result.tfrecords') as f:
        f.write(sample.SerializeToString())

In [44]:
test_dataset = tf.data.TFRecordDataset('robot_reboot_data/testing.tfrecords')
feature_desc = {
    's': tf.io.VarLenFeature(tf.float32)
}
for serialized in test_dataset:
    parsed = tf.io.parse_single_example(serialized, feature_desc)
    s = tf.reshape(tf.sparse.to_dense(parsed['s']), (rows, cols, layers)).numpy()

In [45]:
matrix = s

In [57]:
rows, cols, robots = matrix.shape
maze = matrix[:, :, 0]
np.argwhere(matrix[:, :, 4]==1)

array([], shape=(0, 2), dtype=int64)

In [59]:
robots

9

In [24]:
evaluate(trained_model, s, 0, max_depth=3, playouts=2)

NameError: name 'game' is not defined

In [23]:
history = {
    'history': {
        'value': [0, 1, 1, -1, -1],
        'score': [12, 12, 12, 12]
    }
}
history

{'history': {'value': [0, 1, 1, -1, -1], 'score': [12, 12, 12, 12]}}

In [29]:
import multiprocessing
from multiprocessing import Value
def a_function(ret_value):
    ret_value.value = 3.14

index = Value("d", 0.0)

reader_process = multiprocessing.Process(target=a_function, args=[ret_value])
reader_process.start()
reader_process.join()

print(ret_value.value)


3.14
